In [ ]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
def define_discriminator(in_shape=(28,28,1)):
model = Sequential()
model.add(Conv2D(64, (3,3), strides=(2,2), padding="same", input_shape=in_shape))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.4))
model.add(Conv2D(64, (3,3), strides=(2,2), padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))
opt = Adam(lr=0.0002,beta_1=0.5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
return model
def define_generator(latent_dim):
model = Sequential()
n_node = 128 *7 *7
model.add(Dense(n_node, input_dim=latent_dim))
model.add(LeakyReLU(alpha=0.2))
model.add(Reshape((7,7,128)))
model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(Conv2D(1, (7,7), activation="sigmoid", padding="same"))
return model
def define_gan(g_model, d_model):
d_model.trainable = False
model = Sequential()
model.add(g_model)
model.add(d_model)
opt = Adam(lr=0.0002, beta_1=0.5)
model.compile(loss='binary_crossentropy', optimizer=opt)
return model
# load and prepare mnist training images
def load_real_samples():
(trainX,_),(_,_) = load_data()
X = expand_dims(trainX, axis = -1)
X=X.astype('float32')
X=X/255.0
return X
# select real samples
def generate_real_samples(dataset, n_samples):
ix = randint(0, dataset.shape[0], n_samples)
X = dataset[ix]
y = ones((n_samples,1))
return X,y
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim,n_samples):
x_input = randn(latent_dim * n_samples)
x_input = x_input.reshape(n_samples, latent_dim)
return x_input
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model,latent_dim,n_samples):
x_input = generate_latent_points(latent_dim,n_samples)
X = g_model.predict(x_input)
y = zeros((n_samples,1))
return X,y
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
for i in range(n * n):